#### 1.1 LSTM without Indicators

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load dataset WITHOUT indicators
data_no_ind = pd.read_parquet("./stock_data/Processed_Stock_Data.parquet")

# Define features and target
features = ['Close_Lag_1', 'Close_Lag_2', 'Volume']
target = 'Close'

# Scale data between 0 and 1
scaler = MinMaxScaler()
data_no_ind[features] = scaler.fit_transform(data_no_ind[features])
data_no_ind[target] = scaler.fit_transform(data_no_ind[[target]])

# Convert to numpy arrays for LSTM
X, y = data_no_ind[features].values, data_no_ind[target].values

# Reshape data into 3D (samples, time steps, features)
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Save preprocessed data
np.save("./models/LSTM_X_No_Indicators.npy", X)
np.save("./models/LSTM_y_No_Indicators.npy", y)

print(f"Data Shape for LSTM: {X.shape}, {y.shape}")


Data Shape for LSTM: (3772, 1, 3), (3772,)


In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load preprocessed LSTM data
X = np.load("./models/LSTM_X_No_Indicators.npy")
y = np.load("./models/LSTM_y_No_Indicators.npy")

# Split data (80% train, 20% test)
split = int(0.8 * len(X))
train_X, test_X = X[:split], X[split:]
train_y, test_y = y[:split], y[split:]

# Define LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(1, train_X.shape[2])),
    LSTM(50),
    Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(train_X, train_y, epochs=50, batch_size=16, verbose=1)

# Save trained model
model.save("./models/LSTM_No_Indicators.keras")

Epoch 1/50


c:\Users\naush\main\Resume\jan 2024\feb\New folder\coursewrork\StockPriceAnalyzer\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0195
Epoch 2/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.8429e-05
Epoch 3/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.2115e-05
Epoch 4/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.9228e-05
Epoch 5/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.8553e-05
Epoch 6/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5867e-05
Epoch 7/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.3769e-05
Epoch 8/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2292e-05
Epoch 9/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.9566e-05
Epoch 10/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.7728e-05
Epoch 11/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.1659e-05
Epoch 12/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5514e-05
Epoch 13/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.3699e-05
Epoch 14/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.6618e-05
Epoch 15/50
189/

In [17]:
# Load model
model = tf.keras.models.load_model("./models/LSTM_No_Indicators.keras")

# Make predictions
predictions = model.predict(test_X)

# Inverse transform predictions
predictions = scaler.inverse_transform(predictions)
actuals = scaler.inverse_transform(test_y.reshape(-1, 1))

# Compute MAE and RMSE
mae_lstm = np.mean(np.abs(actuals - predictions))
rmse_lstm = np.sqrt(np.mean((actuals - predictions) ** 2))

print(f"LSTM (No Indicators) - MAE: {mae_lstm:.4f}, RMSE: {rmse_lstm:.4f}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
LSTM (No Indicators) - MAE: 3.0889, RMSE: 3.9580


#### 1.2 LSTM with Indicators

In [18]:
# Load dataset with indicators
data_with_ind = pd.read_parquet("./stock_data/Processed_Stock_Data_with_Indicators.parquet")

# Define features & target
features_with_ind = ['Close_Lag_1', 'Close_Lag_2', 'Volume', 'RSI', 'EMA_10', 'SMA_10', 'MACD']
target = 'Close'

# Scale data between 0 and 1
scaler = MinMaxScaler()
data_with_ind[features_with_ind] = scaler.fit_transform(data_with_ind[features_with_ind])
data_with_ind[target] = scaler.fit_transform(data_with_ind[[target]])

# Convert to numpy arrays
X, y = data_with_ind[features_with_ind].values, data_with_ind[target].values

# Reshape for LSTM
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Save processed data
np.save("./models/LSTM_X_With_Indicators.npy", X)
np.save("./models/LSTM_y_With_Indicators.npy", y)

print(f"LSTM Data Shape (With Indicators): {X.shape}, {y.shape}")


LSTM Data Shape (With Indicators): (3772, 1, 7), (3772,)


In [19]:
# Load preprocessed data
X = np.load("./models/LSTM_X_With_Indicators.npy")
y = np.load("./models/LSTM_y_With_Indicators.npy")

# Split data (80% train, 20% test)
split = int(0.8 * len(X))
train_X, test_X = X[:split], X[split:]
train_y, test_y = y[:split], y[split:]

# Define LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(1, train_X.shape[2])),
    LSTM(50),
    Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(train_X, train_y, epochs=50, batch_size=16, verbose=1)

# Save trained model
model.save("./models/LSTM_With_Indicators.keras")


Epoch 1/50


c:\Users\naush\main\Resume\jan 2024\feb\New folder\coursewrork\StockPriceAnalyzer\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0139
Epoch 2/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.0975e-05
Epoch 3/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.0856e-05
Epoch 4/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5013e-05
Epoch 5/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.6849e-05
Epoch 6/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2561e-05
Epoch 7/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3707e-05
Epoch 8/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2448e-05
Epoch 9/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0501e-05
Epoch 10/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0712e-05
Epoch 11/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3496e-05
Epoch 12/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3546e-05
Epoch 13/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2429e-05
Epoch 14/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2382e-05
Epoch 15/50
189/

In [20]:
# Load model
model = tf.keras.models.load_model("./models/LSTM_With_Indicators.keras")

# Make predictions
predictions = model.predict(test_X)

# Inverse transform predictions
predictions = scaler.inverse_transform(predictions)
actuals = scaler.inverse_transform(test_y.reshape(-1, 1))

# Compute MAE and RMSE
mae_lstm_ind = np.mean(np.abs(actuals - predictions))
rmse_lstm_ind = np.sqrt(np.mean((actuals - predictions) ** 2))

print(f"LSTM (With Indicators) - MAE: {mae_lstm_ind:.4f}, RMSE: {rmse_lstm_ind:.4f}")

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
LSTM (With Indicators) - MAE: 4.2602, RMSE: 4.9136
